In [5]:
# ==== CONFIG (edit) ====
TICKER = "AAPL"
RAW_DIR = "data/raw"

# ==== IMPORTS ====
import os, re, requests, pandas as pd
from pathlib import Path
from datetime import datetime
try:
    from dotenv import load_dotenv; load_dotenv()
except Exception:
    pass

# ==== FETCHERS ====
def fetch_alpha_vantage_daily(ticker: str) -> pd.DataFrame:
    key = os.getenv("ALPHAVANTAGE_API_KEY")
    if not key:
        raise RuntimeError("No ALPHAVANTAGE_API_KEY in .env")
    url = "https://www.alphavantage.co/query"
    params = {
        "function":"TIME_SERIES_DAILY_ADJUSTED",
        "symbol":ticker,
        "outputsize":"full",
        "apikey":key,
        "datatype":"json",
    }
    r = requests.get(url, params=params, timeout=30); r.raise_for_status()
    data = r.json()
    if "Note" in data or "Information" in data:
        raise RuntimeError(data.get("Note") or data.get("Information"))
    ts = data.get("Time Series (Daily)")
    if not ts:
        raise RuntimeError(f"Unexpected AV response keys: {list(data)[:6]}")
    df = (pd.DataFrame(ts).T
            .rename(columns={
                "1. open":"Open","2. high":"High","3. low":"Low","4. close":"Close",
                "5. adjusted close":"Adj Close","6. volume":"Volume"
            })
            .reset_index().rename(columns={"index":"Date"}))
    return df

def fetch_yfinance(ticker: str) -> pd.DataFrame:
    import pandas as pd
    import yfinance as yf
    df = yf.download(
        ticker,
        period="max",
        auto_adjust=False,
        progress=False,
        group_by="column",   # <- avoids ticker-first MultiIndex in most cases
    )
    if df is None or df.empty:
        raise RuntimeError("yfinance returned no data")

    # Flatten any MultiIndex reliably
    if isinstance(df.columns, pd.MultiIndex):
        # If columns look like (TICKER, Field), keep Field
        lvl1 = df.columns.get_level_values(0).unique().tolist()
        lvl2 = df.columns.get_level_values(-1).unique().tolist()
        if set(["Open","High","Low","Close","Adj Close","Volume"]).issubset(set(lvl2)):
            df.columns = df.columns.get_level_values(-1)
        else:
            # generic flatten: join with space
            df.columns = [" ".join([str(x) for x in tup if x not in (None,"")]).strip()
                          for tup in df.columns.to_list()]
    else:
        # some builds prefix columns like "AAPL Open"
        df.columns = [str(c) for c in df.columns]

    df = df.reset_index()
    return df

# ==== STANDARDIZE & VALIDATE (ultra-tolerant) ===
def standardize_and_validate(df: pd.DataFrame) -> pd.DataFrame:
    import re, numpy as np, pandas as pd

    # 1) force string cols & flatten if needed
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if x not in (None, "")]).strip()
                      for tup in df.columns.to_list()]
    else:
        df.columns = [str(c) for c in df.columns]

    # 2) tolerant matcher
    def find_col(cols, *must_have, exclude=None):
        exclude = [e.lower() for e in (exclude or [])]
        for c in cols:
            lc = c.lower()
            if all(m in lc for m in [m.lower() for m in must_have]) and not any(e in lc for e in exclude):
                return c
        return None

    # locate candidates
    col_date  = find_col(df.columns, "date") or find_col(df.columns, "timestamp") or find_col(df.columns, "datetime")
    col_open  = find_col(df.columns, "open")
    col_high  = find_col(df.columns, "high")
    col_low   = find_col(df.columns, "low")
    col_adj   = find_col(df.columns, "adj", "close")
    # key fix: accept things like "Close AAPL", but exclude "Adj Close"
    col_close = find_col(df.columns, "close", exclude=["adj"])

    # fallback regex for CLOSE (handles weird spacing/prefix/suffix)
    if col_close is None:
        cands = [c for c in df.columns if re.search(r"\bclose\b", c, flags=re.I) and "adj" not in c.lower()]
        if cands:
            col_close = cands[0]

    col_vol   = find_col(df.columns, "volume") or find_col(df.columns, "vol")

    # 3) rename to canonical
    rename = {}
    if col_date:  rename[col_date]  = "Date"
    if col_open:  rename[col_open]  = "Open"
    if col_high:  rename[col_high]  = "High"
    if col_low:   rename[col_low]   = "Low"
    if col_close: rename[col_close] = "Close"
    if col_adj:   rename[col_adj]   = "Adj Close"
    if col_vol:   rename[col_vol]   = "Volume"
    df = df.rename(columns=rename)

    # 4) dtypes
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    for c in ["Open","High","Low","Close","Adj Close"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    if "Volume" in df.columns:
        v = pd.to_numeric(df["Volume"], errors="coerce")
        try: df["Volume"] = v.astype("Int64")
        except Exception: df["Volume"] = v

    # 5) validate
    required = ["Date","Open","High","Low","Close","Volume"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        print("DEBUG columns after rename:", list(df.columns))
        raise ValueError(f"Missing required columns: {missing}")
    if df.empty:
        raise ValueError("Empty DataFrame after fetch")
    na = df[required].isna().sum()
    if na.get("Date",0) > 0 or na.get("Close",0) > 0:
        raise ValueError(f"NAs in critical columns:\n{na}")

    # 6) order & sort
    ordered = [c for c in ["Date","Open","High","Low","Close","Adj Close","Volume"] if c in df.columns]
    df = df[ordered + [c for c in df.columns if c not in ordered]].sort_values("Date").reset_index(drop=True)
    return df


# ==== PIPELINE ====
def pull_and_save(ticker: str, raw_dir: str) -> str:
    Path(raw_dir).mkdir(parents=True, exist_ok=True)
    try:
        df = fetch_alpha_vantage_daily(ticker); src = "alphavantage"
    except Exception as e:
        print(f"Alpha Vantage failed → {e}\nFalling back to yfinance.")
        df = fetch_yfinance(ticker); src = "yfinance"

    df = standardize_and_validate(df)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out = Path(raw_dir)/f"{ticker.upper()}_{src}_{ts}.csv"
    df.to_csv(out, index=False)
    print(f" Saved {len(df):,} rows → {out}")
    return str(out)


In [6]:
csv_path = pull_and_save(TICKER, RAW_DIR)
csv_path


Alpha Vantage failed → Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints
Falling back to yfinance.
 Saved 11,265 rows → data/raw/AAPL_yfinance_20250824_004640.csv


'data/raw/AAPL_yfinance_20250824_004640.csv'

# STEP 2 : Scrape a Small Table 

In [7]:
# === CONFIG (edit if you want another table) ===
URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"  # public, permitted
TABLE_ID = "constituents"  # this page's main table id
RAW_DIR = "data/raw"
OUT_BASENAME = "sp500_constituents"  # used in filename
REQUIRED_COLS = ["Symbol", "Security", "GICS Sector", "GICS Sub-Industry"]

# === IMPORTS ===
import os, re, requests, pandas as pd
from bs4 import BeautifulSoup
from pathlib import Path
from datetime import datetime

# === HELPERS ===
def _ensure_dir(p): Path(p).mkdir(parents=True, exist_ok=True)

def _clean_header(s: str) -> str:
    s = re.sub(r"\s+", " ", s or "").strip()
    # common wiki header cleanups
    return (s.replace("Ticker symbol","Symbol")
             .replace("GICS sector","GICS Sector")
             .replace("GICS sub-industry","GICS Sub-Industry"))

def _clean_text(s: str) -> str:
    if s is None: return ""
    s = re.sub(r"\[\d+\]", "", s)      # remove citation [1], [2], ...
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _maybe_numeric(series: pd.Series) -> pd.Series:
    # try coercing strings like "1,234.56" or "12.3%" to numeric
    s = series.astype(str).str.replace(",", "", regex=False).str.replace("%","", regex=False)
    num = pd.to_numeric(s, errors="coerce")
    # Only keep numeric if we got at least some non-null numeric values
    return num if num.notna().sum() >= max(3, int(0.25*len(num))) else series

def _validate_df(df: pd.DataFrame, required_cols=None):
    if df.empty:
        raise ValueError("Parsed table is empty.")
    if required_cols:
        missing = [c for c in required_cols if c not in df.columns]
        if missing:
            raise ValueError(f"Missing required columns: {missing}")
    # Basic NA + dtype sanity on any numeric-looking columns
    numeric_like = [c for c in df.columns if df[c].dtype.kind in "if"]
    if numeric_like:
        na_counts = df[numeric_like].isna().sum()
        # allow some NAs but flag if everything is NA in a numeric column
        fully_na = [c for c in numeric_like if df[c].notna().sum() == 0]
        if fully_na:
            raise ValueError(f"Numeric columns fully NA after parse: {fully_na}")

# === SCRAPER (BeautifulSoup) ===
def scrape_table_to_df(url: str, table_id: str | None = None, table_index: int = 0) -> pd.DataFrame:
    headers = {"User-Agent": "Mozilla/5.0 (educational project; contact=student@example.com)"}
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    # pick the table
    table = None
    if table_id:
        table = soup.find("table", id=table_id)
        if table is None:
            raise RuntimeError(f"No table with id='{table_id}' on page.")
    else:
        tables = soup.find_all("table")
        if not tables:
            raise RuntimeError("No <table> elements on page.")
        table = tables[table_index]

    # parse header
    ths = table.find_all("th")
    header_cells = [ _clean_header(_clean_text(th.get_text())) for th in ths ]

    # If header row spans multiple lines/sections, fall back to first <tr> th/tds
    if not header_cells:
        first_tr = table.find("tr")
        header_cells = [ _clean_header(_clean_text(x.get_text())) for x in first_tr.find_all(["th","td"]) ]

    # parse rows
    rows = []
    for tr in table.find_all("tr"):
        tds = tr.find_all("td")
        if not tds:  # skip header rows
            continue
        row = [ _clean_text(td.get_text()) for td in tds ]
        rows.append(row)

    if not rows:
        raise RuntimeError("Table has no data rows.")

    # align column count
    max_cols = max(len(r) for r in rows)
    cols = header_cells[:max_cols] if header_cells else [f"col_{i}" for i in range(max_cols)]
    rows_padded = [ r + [""]*(max_cols - len(r)) for r in rows ]

    df = pd.DataFrame(rows_padded, columns=cols)

    # de-dup any duplicate column names that wiki sometimes has
    if df.columns.duplicated().any():
        new_cols = []
        seen = {}
        for c in df.columns:
            if c not in seen:
                seen[c] = 0
                new_cols.append(c)
            else:
                seen[c] += 1
                new_cols.append(f"{c}_{seen[c]}")
        df.columns = new_cols

    # Attempt numeric coercion for any columns that look numeric-ish
    for c in df.columns:
        # if majority of cells look like numbers/percentages, try cast
        sample = df[c].astype(str).str.replace(",","", regex=False).str.replace("%","", regex=False)
        looks_numeric = sample.str.fullmatch(r"-?\d+(\.\d+)?").mean() > 0.5
        if looks_numeric:
            df[c] = _maybe_numeric(df[c])

    return df

# === PIPELINE ===
def scrape_validate_save(url: str, table_id: str | None, out_basename: str,
                         raw_dir: str, required_cols=None) -> str:
    _ensure_dir(raw_dir)
    df = scrape_table_to_df(url, table_id)
    # minimal cleanup for this specific table: drop empty columns if any
    df = df.loc[:, [c for c in df.columns if c and c.strip()]]
    _validate_df(df, required_cols=required_cols)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out = Path(raw_dir)/f"{out_basename}_{ts}.csv"
    df.to_csv(out, index=False)
    print(f"✅ Scraped {len(df):,} rows, {df.shape[1]} columns → {out}")
    return str(out)

# === RUN ===
csv_path_table = scrape_validate_save(URL, TABLE_ID, OUT_BASENAME, RAW_DIR, REQUIRED_COLS)
csv_path_table


✅ Scraped 503 rows, 8 columns → data/raw/sp500_constituents_20250824_005648.csv


'data/raw/sp500_constituents_20250824_005648.csv'

# Step 3 — Documentation

## Data Sources (public & permitted)
1) **Price history (primary)**  
   - Source: Yahoo Finance via `yfinance` (fallback)  
   - URL pattern: https://finance.yahoo.com/quote/{TICKER}/history  
   - Chosen Ticker: **AAPL**  
   - Notes: Alpha Vantage (requests) attempted first; endpoint required premium → fallback engaged.

2) **Small table (scrape)**  
   - Source: Wikipedia — S&P 500 constituents  
   - URL: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies  
   - Table ID: `constituents`

## API / Scrape Parameters
**Price Pull**
- `period="max"`, `auto_adjust=False`, `group_by="column"`, `progress=False`
- Canonical columns expected: `Date, Open, High, Low, Close, Adj Close, Volume`

**Scrape**
- HTTP method: `GET` with simple user-agent
- Parser: `BeautifulSoup("html.parser")`
- Table selection: by `id="constituents"` (falls back to first table if a different URL is used)


## Validation Logic

**Price Data (OHLCV)**
- Columns required: `Date, Open, High, Low, Close, Volume`
- Column discovery is tolerant (handles variants like `"Close AAPL"` and MultiIndex).
- Dtype coercion:
  - `Date` → datetime (`errors="coerce"`)
  - `Open, High, Low, Close, Adj Close` → numeric floats
  - `Volume` → integer-like (`Int64`) where possible
- Fail-fast checks:
  - Missing any required column → **error**
  - Empty DataFrame after fetch → **error**
  - Any NA in `Date` or `Close` → **error**
- Tidying: sort by `Date`, canonical column order.

**Scraped Table**
- Table must not be empty.
- Required text columns (for S&P 500 page): `Symbol, Security, GICS Sector, GICS Sub-Industry`
- Numeric-looking columns are coerced; if a column ends up fully NA after coercion → **error**.
- Drop obviously blank columns.


## Secrets & Compliance

- **`.env` is local only** and contains: `ALPHAVANTAGE_API_KEY=...`
- **Committed**: `.env.example` (template only)
- **Git ignore**: `.env` is listed in `.gitignore`

**Paths**
- Raw API CSV → `data/raw/<TICKER>_<source>_<timestamp>.csv`
- Raw scraped CSV → `data/raw/sp500_constituents_<timestamp>.csv`


In [8]:
# This cell verifies we didn't commit secrets.


import subprocess, shlex

def run(cmd):
    print(f"$ {cmd}")
    out = subprocess.check_output(shlex.split(cmd)).decode().strip()
    print(out or "<no output>")
    return out

# 1) Ensure .env is not tracked
tracked = run("git ls-files .env")
assert tracked == "", "❌ .env appears to be tracked!"

# 2) Ensure .env.example IS tracked
tracked_example = run("git ls-files .env.example")
assert tracked_example.endswith(".env.example"), "❌ .env.example is not tracked!"

print("✅ .env is NOT committed, .env.example IS committed.")


$ git ls-files .env
<no output>
$ git ls-files .env.example
<no output>


AssertionError: ❌ .env.example is not tracked!

## Assumptions & Risks

**Assumptions**
- Yahoo Finance historical data is sufficiently accurate for coursework backtesting.
- Wikipedia table structure (`id="constituents"`) remains stable during the assignment window.
- Network access is allowed for educational use (rate limits acceptable).

**Risks**
- **Schema drift**: Yahoo/Wikipedia can change column names or table structure → scraper/standardizer may break.
- **Delistings & symbol changes**: Ticker histories or S&P 500 membership can change; reproducibility depends on timestamped raw snapshots.
- **Partial outages / throttling**: API or site may throttle; fallback to cached raw CSVs if re-runs fail.
- **Timezones**: `Date` normalized by parsing rules; daily bars are treated as naive dates (documented).
